In [1]:
import joblib
import numpy as np
import librosa
from pathlib import Path
from typing import Dict, List, Tuple, Union
from dataclasses import dataclass
import pandas as pd
new_pipe_DecisionTreeClassifier=joblib.load('Model/pipe_DecisionTreeClassifier.pkl')
new_pipe_KNeighborsClassifier=joblib.load('Model/pipe_KNeighborsClassifier.pkl')




In [2]:
import trans
trans.start()

End


In [3]:
TRAINPATH="SimpleData/train"
TESTPATH="SimpleData/test"

In [4]:
@dataclass
class Features:
    sample_rate: int = 22050
    hop_length: int = 220
    n_fft: int = 2048
    n_frames_per_example: int = 1


def load_wav(path: Union[str, Path], sample_rate: int) -> np.ndarray:
    waveform, _ = librosa.load(path, sr=sample_rate)
    return waveform


def extract_features(file_path: Union[str, Path], params: Features) -> List[np.ndarray]:
    waveform = load_wav(file_path, sample_rate=params.sample_rate)

    spectrogram = librosa.feature.melspectrogram(
        y=waveform, n_fft=params.n_fft, hop_length=params.hop_length
    )

    spectrogram = np.log(1e-20 + np.abs(spectrogram**2))

    n_examples = spectrogram.shape[1] // params.n_frames_per_example

    return [
        spectrogram[
            :, i * params.n_frames_per_example : (i + 1) * params.n_frames_per_example
        ].reshape(1, -1)
        for i in range(n_examples)
    ]

In [5]:
def load_folder_data(
    path: Path, train: bool, params: Features
) -> Tuple[List[List[np.ndarray]], List[List[str]], List[Path]]:
    features: List[List[np.ndarray]] = []
    labels: List[List[str]] = []
    file_names: List[Path] = list(path.rglob("*.wav"))

    for file_path in file_names:
        file_features = extract_features(file_path, params)

        features.append(file_features)

        if train:
            class_name = file_path.parent.name
            labels.append([class_name] * len(file_features))

    return features, labels, file_names

In [6]:
@dataclass
class ExpConfig:
    features: Features = Features(
        sample_rate=22050,
        hop_length=220,
        n_fft=2048,
        n_frames_per_example=1,
    )
    random_state: int = 42
    test_size: float = 0.1
    train_path: Path = Path(TRAINPATH)
    test_path: Path = Path(TESTPATH)

config = ExpConfig()

In [18]:
# features, labels, file_names = load_folder_data(
#     path=config.train_path, train=True, params=config.features
# )


In [7]:
class_names = [folder.name for folder in config.train_path.iterdir()]

In [8]:
test_features, _, test_pathes = load_folder_data(
    config.test_path, train=False, params=config.features
)

In [9]:
predictions_DecisionTreeClassifier = [
    class_names[new_pipe_DecisionTreeClassifier.predict_proba(np.vstack(test_feature)).mean(0).argmax()]
    for test_feature in test_features
]


In [10]:
predictions_KNeighborsClassifier = [
    class_names[new_pipe_KNeighborsClassifier.predict_proba(np.vstack(test_feature)).mean(0).argmax()]
    for test_feature in test_features
]

In [19]:
(
    pd.DataFrame(
        {
            "index": [
                test_wav_path.name for test_wav_path in config.test_path.iterdir()
            ],
            "genre": predictions_DecisionTreeClassifier,
        }
    ).to_csv("Prediction/result_DecisionTreeClassifier.csv", index=False,encoding='utf-8_sig')
)

In [20]:
(
    pd.DataFrame(
        {
            "index": [
                test_wav_path.name for test_wav_path in config.test_path.iterdir()
            ],
            "genre": predictions_KNeighborsClassifier,
        }
    ).to_csv("Prediction/result_KNeighborsClassifier.csv", index=False,encoding='utf-8_sig')
)


AttributeError: module 'trans' has no attribute 'endall'